In [4]:
import pandas as pd
from Bio import SeqIO
from Bio.ExPASy import ScanProsite

So I dun goofed and forgot to have you you guys download sequences as well for this next step, below is a script you help you merge the two files automatically, you'll use a similar pattern for the next set of steps which invovles extracting protien domains. 

In [6]:
#Read your blast results csv in
df = pd.read_csv("C:\\Users\\cmgro\\Downloads\\hdeA_genome_feature (7).csv")
#Show the first 5 rows
print(df.head())
#Temp lists to attatch variables 
ids = []
seqs = []
#Seq.IO.parse is. an easy way to read through every gene/prot in a fasta file 
for record in SeqIO.parse("C:\\Users\\cmgro\\Downloads\\hdeA_genome_feature (7).fasta", "fasta"):
    # So the fasta ID line is fig|1240239.3.peg.2328|XFF4834R_chr22840|VBIXanFus78442_2328|
    # but we only need the first two elements ("fig and 1240239.3.peg.2328") to make fig|1240239.3.peg.2328
    # We then use this ID to merge back onto the original dataframe (basically we say if this row has this ID, add the sequence to that row)
    
    #Fancy way of extracting the ID
    ids.append("|".join(record.id.split("|")[0:2]))
    #Fancy way of extracting the sequence 
    seqs.append(str(record.seq))
#making a new dataframe from this info 
seq_df = pd.DataFrame.from_dict({"PATRIC ID": ids, "Sequences":seqs})
print(seq_df.head())
#Mergine back and saving 
df = df.merge(seq_df, on="PATRIC ID")
df.to_csv("/Users/cmgro/Downloads/merged_hdeA.csv")

                               Genome  Genome ID     Accession  \
0   Yokenella regensburgei ATCC 43003  1002368.3  AGCL01000039   
1  Legionella massiliensis strain FF2  1034943.3  CCSB01000001   
2        Legionella massiliensis LegA  1034943.4  CCVW01000001   
3     Commensalibacter intestini A911  1088868.3  AGFR01000009   
4          Budvicia aquatica DSM 5075  1111728.3  ATYS01000003   

                PATRIC ID  RefSeq Locus Tag         Alt Locus Tag  \
0  fig|1002368.3.peg.2678  HMPREF0880_02939  VBIYokReg189340_2678   
1   fig|1034943.3.peg.479        BN59_00473                   NaN   
2   fig|1034943.4.peg.480      BN1094_00473                   NaN   
3  fig|1088868.3.peg.1583         CIN_15750  VBIComInt214715_1583   
4   fig|1111728.3.peg.736               NaN  VBIBudAqu220495_0736   

                                          Feature ID Annotation Feature Type  \
0  PATRIC.1002368.3.AGCL01000039.CDS.216602.21691...     PATRIC          CDS   
1  PATRIC.1034943.3.CCSB0100

In [ ]:
import time
sigs = []

for i in df["Sequences"].values:
    handle = ScanProsite.scan(seq=i, mirror="https://prosite.expasy.org")
    result = ScanProsite.read(handle)
    qq = []
    for i in result:
        print(i)
        qq.append(i["signature_ac"])
    time.sleep(5)
    qq = set(qq)
    sigs.append(qq)
    
df["Prosite results"] = sigs
df.to_csv("/Users/christine/Downloads/hdeA_motifs.csv")

In [ ]:
df
